[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/giswqs/gee-tutorials/blob/master/ImageCollection/ic_filtering.ipynb)

## Wide variety of data available

In [ ]:
#!pip install -U earthengine-api

import ee
import geemap

In [ ]:
# Initialize
# ee.Initialize()

Map = geemap.Map()

## Landsat 8

In [ ]:
roi = ee.Geometry.Point(-111.9555, 40.6761)  # TSOB

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                  .filterDate('2020-09-01', '2020-09-30') \
                  .filterBounds(roi)

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}
Map.setCenter(-111.9555, 40.6761, 12)
Map.addLayer(l8.median(), visParams, 'L8 RGB')

Map

## Sentinel-2

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
                  .filterDate('2020-09-01', '2020-09-30') \
                  .filterBounds(roi) \
                  .sort('CLOUDY_PIXEL_PERCENTAGE')

rgbVis = {
  'min': 0,
  'max': 2500,
  'bands': ['B4', 'B3', 'B2'],
};

Map.addLayer(s2.first(), rgbVis, 'S2 RGB');

print(s2.aggregate_array("CLOUDY_PIXEL_PERCENTAGE").getInfo())
first = s2.first()
# print(geemap.image_props(first).getInfo())

Map

## NAIP 2018

In [ ]:
naip18 = ee.ImageCollection('USDA/NAIP/DOQQ') \
                  .filter(ee.Filter.date('2018-01-01', '2018-12-31'))

trueColor = naip18.select(['R', 'G', 'B'])
trueColorVis = {
  'min': 0.0,
  'max': 255.0,
}

Map.setCenter(-111.9555, 40.6761, 12)
Map.addLayer(trueColor, trueColorVis, 'NAIP 2018')

Map

## SRTM slope and elevation

In [ ]:
srtm = ee.Image('USGS/SRTMGL1_003')
elevation = srtm.select('elevation')
slope = ee.Terrain.slope(elevation)
Map.addLayer(elevation, {'min': 1250, 'max': 4000}, 'SRTM elevation')
Map.addLayer(slope, {'min': 0, 'max': 60}, 'SRTM slope')

Map

## Weather data

### GFS Model Forecast

In [ ]:
Map = geemap.Map()

gfs = ee.ImageCollection('NOAA/GFS0P25') \
                  .filter(ee.Filter.date('2021-04-04', '2021-04-05'))

temperatureAboveGround = gfs.select('temperature_2m_above_ground')
visParams = {
  'min': -40.0,
  'max': 35.0,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}
Map.setCenter(-111.9555, 40.6761, 4);
Map.addLayer(temperatureAboveGround, visParams, 'GFS Temperature')

Map

### Real Time Mesoscale Analysis (Feb 2021 cold outbreak)

In [ ]:
rtma = ee.ImageCollection('NOAA/NWS/RTMA') \
                  .filter(ee.Filter.date('2021-02-13', '2021-02-14'));
temp = rtma.select('TMP');
tempVis = {
  'min': -30,
  'max': 30,
  'palette': ['001137', '01abab', 'e7eb05', 'red'],
}

Map.setCenter(-95, 40, 4);
Map.addLayer(temp, tempVis, 'RTMA Temperature');

Map